# Experiments

Here we showcase 3 types of experiments:

* **Hyperparameter optimization** for training of a model (*using Optuna*) **[not ready yet]**

* **Adversarial Training** of a model for different hyperparameters

* Evaluation of **Adversarial Attacks**


Preconditions:

* you need a ***PL-Module*** that specifies your model class and dataset!

# Training of a Model

* define your hyperparameters in `hparams`
* create your instance of `PLModule`
* define a trainer (e.g., define max epochs, ...)
* fit your model!

In [89]:
print(model.val_results_history[-1]["val_acc"])

0.7172495554238293


# II: Adversarial Training

* same as normal training, but call **`model.setAttack`** 


Today:

* source this out as another experiment class, e.g. AdversarialTrainingExperiment
* pass folder name to initialization
* save: hparams, attack_args, attack_class, train_params
* 

In [ ]:
from utils.TrainHelper import TrainHelper, get_all_models_data, get_model_info

list(get_all_models_data()["M5"].keys())

sorted([ (m["final_val_acc"], m["path"]) for m in get_all_models_data()["M5"].values()])[-5:]

#print(get_model_info("M5_attack_ProjectedGradientDescent_v3"))

# III: Robustness Evaluation

Running a single attack:

In [1]:
from attacks.pgd import ProjectedGradientDescent
attack = ProjectedGradientDescent(model.model, model.train_dataloader(), {"norm":"inf", "epsilon": 10.}, early_stopping=10, device='cuda', save_samples=False)
attack.attack()
attack.report( )

ModuleNotFoundError: No module named 'attacks'

### Documentation
* you define an experiment: an object of class `utils.RobustnessExperiment`.
    * pass a title & description for the experiment (optional)
    * pass different attacks + a list of configs, e.g.    
    
```python

exp_config = [           
              {
               "attack_fn": fast_gradient_attack, 
               "attack_arg": {"norm":["inf"], "epsilon": [0, 0.01, 0.02, 0.05, 0.1, 0.2, 0.5, 1]},
               "meta": {"key_result":"acc", "key_config":"epsilon", "title":"FGSM"}
              }
            ]
```

* all values in `attack_arg` must be in brackets. The attack will the be run for all of the values in each of the lists

* you then can **run** the experiment by passing a **model**
* the following will happen:
    * initializing of the experiment creates a folder in the dir specified in config, named ID_title. Also a pickle file will be saved to the directory.
    * if you run an attack:
        * a sub-folder in that directory will be created for the model.
        * all attacks will be ran
        * for every attack, a plot will be created
        * one json (i.e. one for each model) will be saved to the directory
        * also some samples are stored for each attack.
        
        
Workflow:

* prepare experiment config and use it to initalize experiment
* then add as many models you like
* best: run this not in notebook but as slurm job